In [1]:
import pandas as pd
import ast
import re
from tqdm import tqdm

In [2]:
# Extract player id function using regular expressions
def extract_player_id(entry):
    matches = re.findall(r'\d+\s+(\w+)', entry)
    return matches



In [3]:
def get_on_off_ratings(df, stats):

    def unique_values_from_list_column(dataframe, column_name):
        # Ensure the column contains lists
        if not all(isinstance(value, list) for value in dataframe[column_name]):
            raise ValueError(f"The values in column '{column_name}' are not lists.")

        # Flatten the lists and get unique values
        unique_values = set(item for sublist in dataframe[column_name] for item in sublist)

        return list(unique_values)

    # Get list of all players
    all_players = unique_values_from_list_column(df, 'player_ids')
    
    # get dataframe to store player scoring info
    player_data_df = pd.DataFrame(index =  all_players, columns = ['opoints', 'oscores', 'oscoredon', 'dpoints', 'dscores', 'dscoredon']).fillna(0)
    player_data_df.reset_index(inplace = True)
    player_data_df.rename(columns = {'index': 'player_id'}, inplace = True)    

    # get all teams and create dataframe to store info on teams scoring
    all_teams = list(set(df['AwayTeam']))
    team_data_df = pd.DataFrame(index =  all_teams, columns = ['opoints', 'oscores', 'oscoredon', 'dpoints', 'dscores', 'dscoredon']).fillna(0)
    team_data_df.rename(columns = {'index': 'team'}, inplace = True)    

    # set initial values of vairables
    lp = 0
    lpod = True
    keystr = ''

    # while loop to loop through all play by play data
    for i in tqdm(range(1, len(df))):

        # check if it is a new point and then you will figure out what is the deal with the last point
        if df.loc[i, 'point'] != lp and  df.loc[i, 'event_number'] != 50:

            # reset value of variables
            keystr = ''
            minu=0
            
            # check if the previous point was offense or defense and set key values accordingly
            if lpod:
                keystr+='o'
                keypts='opoints'
            else:
                keystr +='d'
                keypts = 'dpoints'
                
            # figure out which team scored on the previous point and set key values accordingly
            if df.loc[i-1, 'event_number']==22:
                keystr+='scores'
                minu=1
            elif df.loc[i-1, 'event_number']==21:
                keystr+= 'scoredon'
                minu=1
            elif df.loc[max(0, i-2), 'event_number']==22:
                keystr+='scores'
                minu = 2
            elif df.loc[max(0, i-2), 'event_number']==21:
                keystr+= 'scoredon'
                minu = 2
                
            # if a score occoured on the pre
            if keystr in['oscores', 'oscoredon', 'dscores', 'dscoredon']:

    
                
                # get players and team for previous point
                indexes = list(df.loc[i-minu, 'player_ids'])
                # indexes = list(set(player_data_df[player_data_df['player_id'].isin(df.loc[i-minu, 'player_ids'])]['player_id']))
                curteam = df.loc[i-minu, 'HomeTeam'] if df.loc[i-minu, 'team'] == 'homeEvents' else df.loc[i-minu, 'AwayTeam']
                # add scoring and point to player dataframe
                for ind in indexes:
                    player_data_df.loc[player_data_df[player_data_df['player_id'] == ind].index,keystr] += 1
                    player_data_df.loc[player_data_df[player_data_df['player_id'] == ind].index, keypts] += 1
                    
                # add score to team dataframe
                team_data_df.loc[curteam, keystr] += 1
                team_data_df.loc[curteam, keypts] += 1       

            # for the new point check if the team pulled and if they did set them to defense and else offense
            lp = df.loc[i, 'point']
            if df.loc[i, 'event_number'] == 3:
                lpod = False                    
            else:
                lpod = True

        # if there was a timeout with a lineup change reset offense or defense to reality at timeout
        if df.loc[i, 'event_number'] in [40,41]:
            if df.loc[i, 'event_number']  == 40:
                lpod = False
            else:
                lpod = True
                
       
    # calculate the player score rates for offensive and defensive possesions
    player_data_df['oscorerate'] = player_data_df['oscores'] / player_data_df['opoints']
    player_data_df['dscorerate'] = player_data_df['dscores'] / player_data_df['dpoints']
    
    # format team dataframe
    team_data_df.reset_index(inplace = True)
    team_data_df.rename(columns ={'index': 'team'}, inplace = True)
    
    # combine stats and player dataframe
    all_player_info = player_data_df.merge(stats, left_on = 'player_id', right_on = 'playerID', how = 'inner')
    all_player_info.reset_index(inplace = True)

    # calculate the offensive and defensive on off ratings for players
    all_player_info['o_point_on_off_rating'] = all_player_info.apply(lambda row: 100*(row['oscorerate'] - ((team_data_df[team_data_df['team'] == row['teams']]['oscores'].values[0] - row['oscores']) / (team_data_df[team_data_df['team'] == row['teams']]['opoints'].values[0] - row['opoints']))), axis=1)
    all_player_info['d_point_on_off_rating'] = all_player_info.apply(lambda row: 100*(row['dscorerate'] - ((team_data_df[team_data_df['team'] == row['teams']]['dscores'].values[0] - row['dscores']) / (team_data_df[team_data_df['team'] == row['teams']]['dpoints'].values[0] - row['dpoints']))), axis=1)
    all_player_info[['o_point_on_off_rating', 'd_point_on_off_rating']].fillna(0, inplace = True)

    # limit final dataframe to specific columns
    final_ratings = all_player_info[['name', 'player_id','teams','gamesPlayed', 'opoints','oscores', 'dpoints', 'dscores', 'oscorerate', 'dscorerate', 'o_point_on_off_rating', 'd_point_on_off_rating']]

    # get the total pionts and total rating for each player
    final_ratings['total_points'] = final_ratings['opoints'] + final_ratings['dpoints']
    final_ratings = final_ratings[final_ratings['total_points'] > 0]
    final_ratings['total_on_off_rating'] = final_ratings.apply(lambda row: (row['o_point_on_off_rating']*row['opoints'] + row['d_point_on_off_rating']*row['dpoints'])/row['total_points'], axis = 1)
    
    return final_ratings, team_data_df

In [4]:
all_ratings = pd.DataFrame()

for year in [2021,2022,2023,2024]:
    # Dataframe of play by play data
    df = pd.read_csv(str(year)+'playdata.csv')

    # dataframe of all season stats
    stats = pd.read_csv('mixed_model_results_aug_24.csv')

    # Apply the function to each element in the list
    df['player_ids'] = df['lineup'].apply(lambda x: extract_player_id(x))
    df['AwayTeam'] = df['gameID'].str.split('-', expand=True)[3]
    df['HomeTeam'] = df['gameID'].str.split('-', expand=True)[4]

    # get rid of all star game data
    df = df[~df['gameID'].str.contains('allstar')]
    df = df.reset_index(drop = True)

    # select specific season stats and eliminate players that played for multiple teams
    stats = stats[stats['year'] == year]
    stats = stats[~stats['teams'].str.contains(',')]

    ratings, team_df = get_on_off_ratings(df, stats)
    ratings['year'] = year
    
    all_ratings  = pd.concat([all_ratings, ratings], axis = 0)
    
    print('Finished year', year)


100%|██████████| 105379/105379 [04:35<00:00, 382.98it/s]
C:\Users\jacob\AppData\Local\Temp\ipykernel_19336\1521351754.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_player_info[['o_point_on_off_rating', 'd_point_on_off_rating']].fillna(0, inplace = True)
C:\Users\jacob\AppData\Local\Temp\ipykernel_19336\1521351754.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ratings['total_points'] = final_ratings['opoints'] + final_ratings['dpoints']


Finished year 2021


100%|██████████| 127593/127593 [04:48<00:00, 441.55it/s]
C:\Users\jacob\AppData\Local\Temp\ipykernel_19336\1521351754.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_player_info[['o_point_on_off_rating', 'd_point_on_off_rating']].fillna(0, inplace = True)
C:\Users\jacob\AppData\Local\Temp\ipykernel_19336\1521351754.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ratings['total_points'] = final_ratings['opoints'] + final_ratings['dpoints']


Finished year 2022


100%|██████████| 124132/124132 [04:30<00:00, 458.85it/s]
C:\Users\jacob\AppData\Local\Temp\ipykernel_19336\1521351754.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_player_info[['o_point_on_off_rating', 'd_point_on_off_rating']].fillna(0, inplace = True)
C:\Users\jacob\AppData\Local\Temp\ipykernel_19336\1521351754.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ratings['total_points'] = final_ratings['opoints'] + final_ratings['dpoints']


Finished year 2023


100%|██████████| 124416/124416 [04:30<00:00, 459.47it/s]


Finished year 2024


C:\Users\jacob\AppData\Local\Temp\ipykernel_19336\1521351754.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_player_info[['o_point_on_off_rating', 'd_point_on_off_rating']].fillna(0, inplace = True)
C:\Users\jacob\AppData\Local\Temp\ipykernel_19336\1521351754.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ratings['total_points'] = final_ratings['opoints'] + final_ratings['dpoints']


In [23]:
all_ratings[all_ratings['total_points']>=100].sort_values('total_on_off_rating', ascending = False).head(20)

,name,player_id,teams,gamesPlayed,opoints,oscores,dpoints,dscores,oscorerate,dscorerate,o_point_on_off_rating,d_point_on_off_rating,total_points,total_on_off_rating,year
533,Travis Dunn,tdunn,SD,12,234,175,14,4,0.747863,0.285714,39.786325,4.025974,248,37.767595,2024
293,Jack Williams,jwilliams,NY,13,230,181,14,2,0.786957,0.142857,35.838509,-14.355062,244,32.958550,2024
260,Alec Wilson Holliday,awilsonho,DAL,12,253,171,7,1,0.675889,0.142857,30.088933,0.078064,260,29.280948,2024
572,Pawel Janas,pjanas,LA,11,210,159,5,2,0.757143,0.400000,28.095238,25.500000,215,28.034884,2024
732,Austin Taylor,ataylor,ATL,13,222,185,10,3,0.833333,0.300000,28.160920,0.711297,232,26.977746,2024
385,Kyle Henke,khenke,ATX,13,222,172,22,9,0.774775,0.409091,27.477477,9.351714,244,25.843187,2024
326,Elijah Long,elong,CAR,16,272,217,5,1,0.797794,0.200000,26.446078,-11.864407,277,25.754553,2024
849,Ryan Osgar,rosgar,NY,15,218,203,49,16,0.931193,0.326531,32.008155,-3.034671,267,25.577075,2022
181,Evan Magsig,emagsig,OAK,13,236,187,11,1,0.792373,0.090909,27.063375,-15.151515,247,25.183360,2024
384,Quentin Bonnaud,qbonnaud,MTL,12,200,141,5,3,0.705000,0.600000,24.071429,36.966292,205,24.385937,2024


In [6]:
stats.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'playerID', 'name', 'gamesPlayed',
       'scores', 'assists', 'goals', 'plusMinus', 'completions',
       'completionPercentage', 'hockeyAssists', 'throwaways', 'stalls',
       'drops', 'blocks', 'callahans', 'pulls', 'teams', 'year',
       'pointsPlayed', 'oPointsPlayed', 'dPointsPlayed', 'minutesPlayed',
       'possessions', 'oEfficiency', 'yardsTotal', 'yardsThrown',
       'yardsReceived', 'hucksCompleted', 'huckPercentage', 'oeff_rating',
       'goal_rating', 'assist_rating', 'block_rating', 'composite_rating'],
      dtype='object')

In [38]:
year_stats = pd.read_csv('player_yearly_stats_aug_24.csv')
year_stats = year_stats[year_stats['year'] >=2021]
year_stats = year_stats.merge(all_ratings[['player_id','year', 'oscorerate', 'dscorerate','o_point_on_off_rating', 'd_point_on_off_rating', 'total_on_off_rating']],
                   left_on = ['playerID', 'year'], right_on = ['player_id', 'year'], how = 'left')


In [40]:
pos_info = pd.read_csv('career_stats_w_ratings_aug_24.csv')
year_stats = year_stats.merge(pos_info[['playerID', 'position']], on = 'playerID', how = 'left')

In [43]:
year_stats.to_csv('final_yearly_df_aug_24.csv')